In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, datetime, torch
from typing import *
from recstudio.utils import *
import torch.nn as nn

In [ ]:
import recstudio.eval as eval
model, dataset = 'Test10', 'amazon-beauty'
model_class, model_conf = get_model(model)
model = model_class(model_conf)
dataset_class = model_class._get_dataset_class()

data_conf = {}

data_conf.update(model_conf['data'])    # update model-specified config
datasets = dataset_class(name=dataset, config=data_conf).build(**model_conf['data'])
train_data = datasets[0]


import logging
logger = get_logger('./test.log')
logger.setLevel(logging.ERROR)
model.logger = logging.getLogger('recstudio')

model._init_model(train_data)
model._init_parameter()
model.run_mode = 'light'
model.val_check = True
val_metrics = model.config['eval']['val_metrics']
model.val_metric = next(iter(val_metrics)) if isinstance(val_metrics, list) else val_metrics
cutoff = model.config['eval']['cutoff']
cutoffs = cutoff if isinstance(cutoff, list) else [cutoff]
if len(eval.get_rank_metrics(model.val_metric)) > 0:
    model.val_metric += '@' + str(cutoffs[0])
model.callback = model._get_callback(train_data.name)
model._accelerate()

In [ ]:
save_dir = model.config['eval']['save_path']
model.save_dir = save_dir
ckpt = './saved/Test10/amazon-beauty/2023-05-16-15-25-14-166515.ckpt'
model.ckpt_path = './Test10/amazon-beauty/2023-05-16-15-25-14-166515.ckpt'
model.load_checkpoint(ckpt)
model.trainloaders = model._get_train_loaders(train_data)
model.optimizers = model._get_optimizers()

In [ ]:
model.config['model']['cl_weight'] = 0.5

In [ ]:
for g in model.optimizers[0]['optimizer'].param_groups:
    g['lr'] = 0.001

In [ ]:
model.finetune = True
out = model.evaluate(datasets[-1])

In [ ]:
out